In [1]:
from mypydrive.auth import GoogleAuth
from mypydrive.drive import GoogleDrive
from pprint import pprint
import pyAesCrypt
import io
import os

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1041234379035-ua0eveg1b0rb811hfmnj9n0m348a632h.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


https://console.cloud.google.com/apis/credentials?authuser=1&folder=&organizationId=&project=quickstart-1579349870305

In [2]:
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
    print('Title: %s, ID: %s, mime: %s' % (file['title'], file['id'], file['mimeType']))
    # Get the folder ID that you want
    if(file['title'] == "To Share"):
        fileID = file['id']

Title: py_backup, ID: 1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6, mime: application/vnd.google-apps.folder
Title: test.jpg.ciph, ID: 14NCUe27rLpinRWTg_nbw75raCvqoMXHa, mime: application/octet-stream
Title: ctest.ciph, ID: 15P9bSCfd7bC8rZaTTxWu8AnALRBOcrTP, mime: application/octet-stream
Title: KMS_Tools.zip, ID: 11jlfTYrjc2aanZlul6S9PP6QGOvgeAkg, mime: application/zip
Title: other, ID: 15UZpLHZ-koagoj_T7RDYStcpKNXFcC5h, mime: application/vnd.google-apps.folder
Title: Backup, ID: 1prgjfxfDBJOL_FxhDekqEk7Z1lXoEGGO, mime: application/vnd.google-apps.folder
Title: orbi, ID: 1Qxre8vhr4qIsP2Sw2Qo8SM8SAzU9XQjm, mime: application/vnd.google-apps.folder
Title: Share, ID: 1d5WSWmw8yJ-q4YwGq8MgeuMUM50NFXZP, mime: application/vnd.google-apps.folder


In [ ]:
# Create GoogleDriveFile instance with title 'Hello.txt'.
file1 = drive.CreateFile()
# file1.SetContentFile('test.jpg')
file1.SetContentString(b'qweradf')
file1.Upload() # Upload the file.

In [ ]:
# file1.SetContentString(img.dencode())
file1 = drive.CreateFile()
file1.SetContentBinary(open("test.jpg", "rb"), "ani.jpg")
file1.Upload() # Upload the file.

In [ ]:
def SetContentBinary(self, data, filename):
    """Set content of this file from a file.
    Opens the file specified by this method.
    Will be read, uploaded, and closed by Upload() method.
    Sets metadata 'title' and 'mimeType' automatically if not specified.
    :param filename: name of the file to be uploaded.
    :type filename: str.
    """
    self.content = data
    if self.get('title') is None:
        self['title'] = filename
    if self.get('mimeType') is None:
        self['mimeType'] = mimetypes.guess_type(filename)[0]

def GetContentBinary(self, filename, mimetype=None, remove_bom=False):
    """Save content of this file as a local file Dimon.

    :param filename: name of the file to write to.
    :type filename: str
    :param mimetype: mimeType of the file.
    :type mimetype: str
    :param remove_bom: Whether to remove the byte order marking.
    :type remove_bom: bool
    :raises: ApiRequestError, FileNotUploadedError, FileNotDownloadableError
    """
    if self.content is None or \
                    type(self.content) is not io.BytesIO or \
                    self.has_bom == remove_bom:
        self.FetchContent(mimetype, remove_bom)
    return self.content.getvalue()

### Test encription IO

In [ ]:
import pyAesCrypt
import io

bufferSize = 64 * 1024
password = "foopassword"

# binary data to be encrypted
pbdata = b"This is binary plaintext"

# input plaintext binary stream
fIn = io.BytesIO(pbdata)

# initialize ciphertext binary stream
fCiph = io.BytesIO()

# initialize decrypted binary stream
fDec = io.BytesIO()

# encrypt stream
pyAesCrypt.encryptStream(fIn, fCiph, password, bufferSize)

# print encrypted data
print("This is the ciphertext:\n" + str(fCiph.getvalue()) +  f" {len(str(fCiph.getvalue()))}")

# get ciphertext length
ctlen = len(fCiph.getvalue())

# go back to the start of the ciphertext stream
fCiph.seek(0)

# decrypt stream
pyAesCrypt.decryptStream(fCiph, fDec, password, bufferSize, ctlen)

# print decrypted data
print("Decrypted data:\n" + str(fDec.getvalue()))

In [ ]:
len(b"AES\x02\x00\x00\x1bCREATED_BY\x00pyAesCrypt 0.4.3") + 1024

In [ ]:
file1 = drive.CreateFile()
file1.SetContentBinary(fCiph, "ctest.ciph")
file1.Upload() # Upload the file.

In [ ]:
# decrypt stream
fCiphFile = open("ctest.ciph", "rb")
pyAesCrypt.decryptStream(fCiphFile, fDec, password, bufferSize, ctlen)

# print decrypted data
print("Decrypted data:\n" + str(fDec.getvalue()))

In [ ]:
# old 
def encript_and_upload(fname, bufferSize = 64 * 1024, password = "foopassword"):
    fIn = open(fname, "rb")
    fCiph = io.BytesIO()
    pyAesCrypt.encryptStream(fIn, fCiph, password, bufferSize)
    
    file1 = drive.CreateFile()
    file1.SetContentBinary(fCiph, f"{fname}.ciph")
    file1.Upload() # Upload the file.
    
    fIn.close()
    return fCiph

efile = encript_and_upload("test.jpg")
print(len(str(efile.getvalue())))
len(str(open("test.jpg", "rb").read()))

In [ ]:
def decript_and_save(fname, bufferSize = 64 * 1024, password = "foopassword"):
    pyAesCrypt.decryptFile(fname, fname.replace(".ciph", ""), password, bufferSize)



"1M5j2NCzg-o-iq2r6JakvEeo0Fwh6GxG-"
decript_stream_and_save('asd.txt', "1M5j2NCzg-o-iq2r6JakvEeo0Fwh6GxG-")

# dfile = drive.CreateFile({'id': "1M5j2NCzg-o-iq2r6JakvEeo0Fwh6GxG-"})
# fCiph = dfile.GetContentBinary("asd")
# print(fCiph)
# decript_and_save("test2.jpg.ciph")

In [ ]:
len(fCiph)

### Useful functions

In [3]:
def driveListFolder(parent):
    filelist=[]
    file_list = drive.ListFile({'q': f"'{parent}' in parents and trashed=false"}).GetList()
    for f in file_list:
        if f['mimeType']=='application/vnd.google-apps.folder': # if folder
            filelist.append({"id": f['id'],
                             "isfolder": True,
                             "title": f['title'],
                             "list": driveListFolder(f['id'])
                            })
        else:
            filelist.append({"id": f["id"],
                             "title": f['title'], 
                             "isfolder": False,
#                              "title1": f['alternateLink']
                            })
    return filelist

def localListFolder(parent, level=0, verbose=False,
                   ignore=['.git', ".ipynb_checkpoints", "__pycache__"]):
    cwd = os.getcwd()  # restore cwd after recursion is finished
    os.chdir(parent) 
    indent = '    '
    folder_dict={"isfolder": True, 
                 "title": parent,
                 "children": []}
    file_list = [asd for asd in os.listdir(".") if asd not in ignore]
    for f in file_list:
        if os.path.isdir(f): # if folder
            if verbose: print(f"{indent*level}{parent}: folder {f}")
            folder_dict['children'].append(localListFolder(f, level=level+1, verbose=verbose))
        else:
            if verbose: print(f"{indent*level}{parent}: file   {f}")
            folder_dict['children'].append({"isfolder": False, 
                                            "title": f})

    os.chdir(cwd)
    return folder_dict

# pprint(localListFolder('.'))
# dlist = driveListFolder(asd.root_id)
# pprint(dlist)
# os.getcwd()

In [4]:
# dlist = driveListFolder(asd.root_id)

def driveList2localDict(dlist, id, parent, level=0, verbose=True, indent = "|---"):
    folder_dict={"isfolder": True, 
                 "id": id,
                 "title": parent,
                 "children": []}
    for f in dlist:
        if f["isfolder"]:
            if verbose: print(f"{indent*level}{parent}: FOLDER {f['title']}")
            folder_dict['children'].append(driveList2localDict(f['list'], f['id'], f['title'], level+1, verbose, indent))
        else:
            if verbose: print(f"{indent*level}{parent}: FILE   {f['title']}")
            folder_dict['children'].append({"isfolder": False, 
                                            "id": f["id"],
                                            "title": f['title']})
    return folder_dict
            
# pprint(driveList2localDict(dlist, "Asd", "py_backup", verbose=False))

In [5]:
def encript_and_upload(fname, parent_id, bufferSize = 64 * 1024, password = "foopassword"):
    fIn = open(fname, "rb")
    fCiph = io.BytesIO()
    pyAesCrypt.encryptStream(fIn, fCiph, password, bufferSize)
    
    file = drive.CreateFile({'title': fname, 
                             'parents':[{'id': parent_id}]
                             })
    file.SetContentBinary(fCiph)
    file.Upload() # Upload the file.
    
    fIn.close()
    return file['id']

def decript_stream_and_save(drive_file_id, bufferSize = 64 * 1024, password = "foopassword"):
    fDec = io.BytesIO()
    dfile = drive.CreateFile({'id': drive_file_id})
    fCiph = io.BytesIO(dfile.GetContentBinary())
    pyAesCrypt.decryptStream(fCiph, fDec, password, bufferSize, len(fCiph.getvalue()))
    with open(dfile['title'], 'wb') as f:
        f.write(fDec.getvalue())

In [6]:
def mkdir(folder_name):
    try: 
        os.mkdir(folder_name)
    except FileExistsError:
        pass

### Drive folders

In [7]:
class DriveHandler:
    
    def __init__(self, password = "foopassword", bufferSize = 64 * 1024):
        self.root_id = "1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6"
#         self.root = drive.CreateFile({'id': self.root_id})
        self.cache = None
        self.bufferSize = bufferSize
        self.password = password
    
    def create_folder(self, child_name=None, parent_id=None):
        if parent_id is None:
            parent_id = self.root_id
        if child_name is None:
            child_name = "New Folder"
        child_folder = drive.CreateFile({'title': child_name, 
                                         'parents':[{'id':parent_id}], 
                                         'mimeType' : 'application/vnd.google-apps.folder'})
        child_folder.Upload()
        return child_folder['id']
    
    def create_encrypted_file(self, fname, parent_id):
        if parent_id is None:
            parent_id = self.root_id
        return encript_and_upload(fname, parent_id, self.bufferSize, self.password)
    
    def upload_folder_tree(self, tree, drive_id=None, level=0, verbose=True, indent = "|---"):
        cwd = os.getcwd()  # restore cwd after recursion is finished
        parent_folder = tree['title']
        os.chdir(parent_folder) 
            
        for f in tree['children']:
            if f['isfolder']:
                f['id'] = self.create_folder(f['title'], drive_id)
                if verbose: print(f"{indent*level}> {parent_folder}: FOLDER {f['title']} \t\t {f['id']}")
                self.upload_folder_tree(f, f['id'], level+1, verbose, indent)
            else:
                f['id'] = self.create_encrypted_file(f['title'], drive_id)
                if verbose: print(f"{indent*level}> {parent_folder}: FILE {f['title']} \t\t {f['id']}")
        
        os.chdir(cwd)
        return tree
    
    def download_folder_tree(self, fsave, tree, ignore=[], level=0, verbose=True, indent = "|---"):
        cwd = os.getcwd()  # restore cwd after recursion is finished
        parent_folder = tree['title']
        os.chdir(fsave) 
        
        for f in tree['children']:
            if f['isfolder']:
                mkdir(f['title'])
                if verbose: print(f"{indent*level}> {parent_folder}: FOLDER {f['title']} \t\t {f['id']}")
                self.download_folder_tree(f['title'], f, verbose=verbose, indent=indent, level=level+1)
            else:
                decript_stream_and_save(f['id'], self.bufferSize, self.password)
                if verbose: print(f"{indent*level}> {parent_folder}: FILE {f['title']} \t\t {f['id']}")
        
        os.chdir(cwd)
    
    def scan_drive_tree(self, path_id):
        dlist = driveListFolder(path_id)
        return driveList2localDict(dlist, path_id, "root", verbose=False)
        
    def scan_local_folder_tree(self, path):
        return localListFolder(path)

os.chdir("D:\\Code_Base\\pyDrive")
asd = DriveHandler()
dfiles = asd.scan_drive_tree(asd.root_id)
# pprint(dfiles)


asd.download_folder_tree('download', dfiles)
# files = localListFolder('.')
# pprint(files)
# dfiles = asd.upload_folder_tree(files)
# pprint(dfiles)

> root: FOLDER pyAesCrypt 		 15yXQqX35rt8RG9z2ZjyAqaSBFv3g0OGu
> pyAesCrypt: FILE __init__.py 		 1VzrPjr7F8naNOsRbGVbXfs8GlhB9Cpra
> pyAesCrypt: FILE test_crypto.py 		 10b7s0qP3iBoWoz4ljdsawzPDiHuaDIC9
> pyAesCrypt: FILE crypto.py 		 1UXiuvpIZTIbMQd1vbuxhqyBl2cplG-Xg
> root: FOLDER mypydrive 		 1AahvtKsON23QCpWy9V3sFfu0iogoOvdL
> mypydrive: FILE __init__.py 		 1FtE4ciUO6P3z7v4g82mh11QtXug0MiYm
> mypydrive: FOLDER test 		 18_6zahp24csAkQbO-dFwJ2sy93Kvals9
> test: FILE __init__.py 		 1U3vGOrlntnzrCQQDAoy8VVCJk-muWCX-
> test: FILE test_util.py 		 1EUI0UPqmP9m74kWzrIS8QiNGNL8HDk35
> test: FILE test_oauth.py 		 1JbNZXsFjOda-jl5_FJgCg1lizrgGyF32
> test: FILE test_filelist.py 		 1Ax7XUelLo5aPpqY59D8gHSVQbxZ3UGE0
> test: FILE test_file.py 		 11rLIvOokImFq5oATJjEFgKJE4lPUh_Hr
> test: FILE test_drive.py 		 1Wy3eIvt6Y1zrrr-ldIyQcP8_5yYgtf1S
> test: FILE test_apiattr.py 		 1S6VVowEZOLec6dHkrl7BufemuhIMIEm3
> test: FOLDER settings 		 1sjOdzmXHjKanp4jrXmfZtpJG5-nzFO-B
> settings: FILE test6.yaml 		 

In [ ]:
print(parent_id)
file2 = drive.CreateFile({'id': parent_id})
print('title: %s, mimeType: %s' % (file2['title'], file2['mimeType']))

In [ ]:

# driveListFolder('1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6')
os.chdir("D:\\Code_Base\\pyDrive")
asdd = localListFolder('.', verbose=True)

In [ ]:
# l = os.listdir("./.ipynb_checkpoints")
# os.chdir("..")
# print([d for d in l if os.path.isdir(d)])
# print([d for d in l if not os.path.isdir(d)])
os.getcwd() 